# Домашняя работа №3. 

_Екатерина Волошина_

### Импортируем модули и скачиваем Маллет для работы в Гугл Колабе: 

In [33]:
import os       #importing os to set environment variable
def install_java():
  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null      #install openjdk
  os.environ["JAVA_HOME"] = "usr/lib/jvm/java-8-openjdk-amd64"     #set environment variable
  !java -version       #check java version
install_java()

openjdk version "11.0.8" 2020-07-14
OpenJDK Runtime Environment (build 11.0.8+10-post-Ubuntu-0ubuntu118.04.1)
OpenJDK 64-Bit Server VM (build 11.0.8+10-post-Ubuntu-0ubuntu118.04.1, mixed mode, sharing)


In [34]:
!wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
!unzip mallet-2.0.8.zip

--2020-10-19 09:59:16--  http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
Resolving mallet.cs.umass.edu (mallet.cs.umass.edu)... 128.119.246.70
Connecting to mallet.cs.umass.edu (mallet.cs.umass.edu)|128.119.246.70|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16184794 (15M) [application/zip]
Saving to: ‘mallet-2.0.8.zip.2’

mallet-2.0.8.zip.2  100%[===================>]  15.43M  18.8MB/s    in 0.8s    

2020-10-19 09:59:17 (18.8 MB/s) - ‘mallet-2.0.8.zip.2’ saved [16184794/16184794]

Archive:  mallet-2.0.8.zip
replace mallet-2.0.8/bin/classifier2info? [y]es, [n]o, [A]ll, [N]one, [r]ename: All
  inflating: mallet-2.0.8/bin/classifier2info  
  inflating: mallet-2.0.8/bin/csv2classify  
  inflating: mallet-2.0.8/bin/csv2vectors  
  inflating: mallet-2.0.8/bin/mallet  
  inflating: mallet-2.0.8/bin/mallet.bat  
  inflating: mallet-2.0.8/bin/mallethon  
  inflating: mallet-2.0.8/bin/prepend-license.sh  
  inflating: mallet-2.0.8/bin/svmlight2vectors  
  inflating

In [36]:
import zipfile
from google.colab import files

uploaded = files.upload()

Saving mallet-2.0.8.zip to mallet-2.0.8 (2).zip


In [37]:
path_to_zip_file = list(uploaded.keys())[0]

print ('Extracting',path_to_zip_file)

with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
  zip_ref.extractall('.')

print()
print('Here is a directory listing (you should see a directory with your corpus):')
!ls -l

Extracting mallet-2.0.8.zip

Here is a directory listing (you should see a directory with your corpus):
total 79048
drwxr-xr-x 10 root root     4096 Oct 19 10:03  mallet-2.0.8
-rw-r--r--  1 root root 16184794 Oct 19 09:41 'mallet-2.0.8 (1).zip'
-rw-r--r--  1 root root 16184794 Oct 19 10:07 'mallet-2.0.8 (2).zip'
-rw-r--r--  1 root root 16184794 Oct 19 09:21  mallet-2.0.8.zip
-rw-r--r--  1 root root 16184794 Nov 14  2016  mallet-2.0.8.zip.1
-rw-r--r--  1 root root 16184794 Nov 14  2016  mallet-2.0.8.zip.2
drwxr-xr-x  1 root root     4096 Oct 14 16:31  sample_data


In [38]:
os.environ['MALLET_HOME'] = '/content/mallet-2.0.8'
mallet_path = '/content/mallet-2.0.8/bin/mallet' # you should NOT need to change this 

### Импорт нужных модулей

In [39]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [40]:
import nltk; nltk.download('stopwords')


! python -m spacy download en

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [41]:
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)


In [42]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [52]:
from tqdm.auto import tqdm

###  Предобработка данных

In [43]:
df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
print(df.target_names.unique())
df.head()

['rec.autos' 'comp.sys.mac.hardware' 'comp.graphics' 'sci.space'
 'talk.politics.guns' 'sci.med' 'comp.sys.ibm.pc.hardware'
 'comp.os.ms-windows.misc' 'rec.motorcycles' 'talk.religion.misc'
 'misc.forsale' 'alt.atheism' 'sci.electronics' 'comp.windows.x'
 'rec.sport.hockey' 'rec.sport.baseball' 'soc.religion.christian'
 'talk.politics.mideast' 'talk.politics.misc' 'sci.crypt']


,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,comp.graphics
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space


In [44]:
# Convert to list
data = df.content.values.tolist()
# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]
# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]
pprint(data[:1])


<input>:4: DeprecationWarning: invalid escape sequence \S
<input>:4: DeprecationWarning: invalid escape sequence \S
<input>:4: DeprecationWarning: invalid escape sequence \S
<input>:4: DeprecationWarning: invalid escape sequence \S
<input>:4: DeprecationWarning: invalid escape sequence \S
<input>:4: DeprecationWarning: invalid escape sequence \S
<input>:4: DeprecationWarning: invalid escape sequence \S
<input>:6: DeprecationWarning: invalid escape sequence \s
<input>:4: DeprecationWarning: invalid escape sequence \S
<input>:6: DeprecationWarning: invalid escape sequence \s
<input>:4: DeprecationWarning: invalid escape sequence \S
<input>:6: DeprecationWarning: invalid escape sequence \s
<input>:4: DeprecationWarning: invalid escape sequence \S
<input>:6: DeprecationWarning: invalid escape sequence \s
<input>:4: DeprecationWarning: invalid escape sequence \S
<input>:6: DeprecationWarning: invalid escape sequence \s
<input>:4: DeprecationWarning: invalid escape sequence \S
<input>:6: Dep

['From: (wheres my thing) Subject: WHAT car is this!? Nntp-Posting-Host: '
 'rac3.wam.umd.edu Organization: University of Maryland, College Park Lines: '
 '15 I was wondering if anyone out there could enlighten me on this car I saw '
 'the other day. It was a 2-door sports car, looked to be from the late 60s/ '
 'early 70s. It was called a Bricklin. The doors were really small. In '
 'addition, the front bumper was separate from the rest of the body. This is '
 'all I know. If anyone can tellme a model name, engine specs, years of '
 'production, where this car is made, history, or whatever info you have on '
 'this funky looking car, please e-mail. Thanks, - IL ---- brought to you by '
 'your neighborhood Lerxst ---- ']


In [45]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
data_words = list(sent_to_words(data))
print(data_words[:1])


[['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'edu', 'organization', 'university', 'of', 'maryland', 'college', 'park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']]


In [46]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp_posting_host', 'rac_wam_umd_edu', 'organization', 'university', 'of', 'maryland_college_park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front_bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']


In [47]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [48]:
data_words_nostops = remove_stopwords(data_words)
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)

nlp = spacy.load('en_core_web_sm')
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
print(data_lemmatized[:1])

[['where', 'thing', 'car', 'nntp_poste', 'host', 'park', 'line', 'wonder', 'could', 'enlighten', 'car', 'see', 'day', 'door', 'sport', 'car', 'look', 'late', 'early', 'call', 'bricklin', 'door', 'really', 'small', 'addition', 'separate', 'rest', 'body', 'know', 'tellme', 'model', 'name', 'engine', 'year', 'production', 'car', 'make', 'history', 'info', 'funky', 'look', 'car', 'mail', 'thank', 'bring', 'neighborhood', 'lerxst']]


In [49]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)
# Create Corpus
texts = data_lemmatized
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1])


[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 5), (6, 1), (7, 1), (8, 2), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 2), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1)]]


### Попробуем натренировать Маллет на данных корпуса

In [50]:
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [51]:
pprint(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

[(10,
  [('drive', 0.03814996884303815),
   ('card', 0.02931062337002931),
   ('system', 0.02541023333102541),
   ('driver', 0.020263564818020264),
   ('bit', 0.016778600937016778),
   ('disk', 0.014839945533014839),
   ('scsi', 0.013962934755013963),
   ('speed', 0.013501350135013501),
   ('monitor', 0.013362874749013362),
   ('memory', 0.013178240901013178)]),
 (17,
  [('gun', 0.023977247042868983),
   ('make', 0.014936336924583741),
   ('money', 0.011847359300836285),
   ('year', 0.01182852407142319),
   ('job', 0.010660739847811346),
   ('government', 0.01058539893015897),
   ('work', 0.009605967000678069),
   ('pay', 0.008438182777066225),
   ('weapon', 0.007948466812325774),
   ('people', 0.007383409929932946)]),
 (18,
  [('question', 0.01920157019590416),
   ('exist', 0.01727585823797356),
   ('claim', 0.014257675073140021),
   ('reason', 0.013850312928193163),
   ('true', 0.013646631855719734),
   ('argument', 0.012998555716031552),
   ('point', 0.011739436358923083),
   ('evid

### Ищем с помощью функции оптимальное количество топиков (проверяем качество с помощью Coherence Score)

In [59]:
def find_optimal_number(first, last, step, mallet_path, corpus, id2word, data_lemmatized):
  coherence_dict = {}
  for i in tqdm(range(first, last, step)):
    ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=i, id2word=id2word)
    coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    coherence_ldamallet = coherence_model_ldamallet.get_coherence()
    coherence_dict[i] = coherence_ldamallet
  coh_dict = sorted(list(coherence_dict.items()), key=lambda i:-i[1])
  return coh_dict[0][0]

In [60]:
opt_number = find_optimal_number(1, 50, 10, mallet_path, corpus, id2word, data_lemmatized)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


### Тренируем модель с оптимальным количеством топиков:

In [61]:
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=opt_number, id2word=id2word)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


### Находим широкий топик с помощью функции:

In [ ]:
def find_main_topic(ldamallet, texts):
  text_to_topic = {}
  for num, i in enumerate(texts):
    topic_freq = {}
    for a in ldamallet.show_topics(formatted=False):
      keywords = dict(a[1])
      for word in i:
        if word in keywords.keys() and word in topic_freq:
          topic_freq[a[0]] += keywords[word]
        elif word in keywords.keys():
          topic_freq[a[0]] = keywords[word]
    topic_freq = sorted(list(topic_freq.items()), key=lambda i:-i[1])
    if topic_freq != []:
      text_to_topic[num] = topic_freq[0][0]
    else:
      text_to_topic[num] = None
  return text_to_topic

In [62]:
text_to_topic = find_main_topic(ldamallet, texts)

### Tf-Idf
С помощью TfidfVectorizer из sklearn считаем td-idf внутри текстов с одним широким топиком. Записываем в датафрейм текст, его широкий топик и пять слов с самым высоким tf-idf

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

tf_idf_vectorize = TfidfVectorizer(use_idf=True)

In [ ]:
def find_tf_idf(ldamallet, texts, data, text_to_topic):
  df = pd.DataFrame(columns=['texts', 'topic', 'keywords'])
  for t in ldamallet.show_topics(formatted=False):
    topic = t[0]
    corpora = []
    texts_id = []
    for num, i in enumerate(texts):
      if text_to_topic[num] == topic:
        corpora.append(' '.join(i))
        texts_id.append(num)
    vectors = tf_idf_vectorize.fit_transform(corpora)
    for i, num in enumerate(texts_id):
      doc_vectors = vectors[i]
      df_v = pd.DataFrame(doc_vectors.T.todense(), index=tf_idf_vectorize.get_feature_names(), columns=["tfidf"]) 
      keywords = list(df_v.sort_values(by=["tfidf"],ascending=False).index[:5])
      df = df.append({'texts':data[num], 'topic':topic, 'keywords': ', '.join(keywords)}, ignore_index=True)
  return df

In [63]:
tf_idf = find_tf_idf(ldamallet, texts, data, text_to_topic)

In [64]:
tf_idf.head()

,texts,topic,keywords
0,Subject: Re: NHLPA poll (partial stats/results...,17,"goalie, anyway, capable, partial, stat"
1,From: (Keith Allan Schneider) Subject: Re: Kei...,17,"religion, tool, say, understand, motto"
2,From: (Henry Spencer) Subject: Re: HST Servici...,17,"reboost, do, thruster, boost, contamination"
3,From: (Harmon Sommer) Subject: Re: Counterstee...,17,"leave, rein, horse, go, counter"
4,From: (Maddi Hausmann) Subject: Re: Amusing at...,17,"honest, thing, post, exist, make"


### Coherence Score

Подсчет Coherence Score состоит из нескольких этапов: датасет делится на набор пар подмножеств слов S. Затем вычисляются вероятности слов на основе заданного корпуса. И набор подмножеств S, и вычисленные вероятности нужны для того, чтобы посчитать согласования между парами подмножеств S.

Для вычисления Согласованности Темы используются две метрики: внутренняя метрика и внешняя метрика. 

Внутренняя метрика сравнивает, как соотносится слово с предыдущим и последующим словом, пожтому нам нужен упорядоченный набор слов. Он использутся как функция попарной оценки в виде условной логарифмической вероятности со сглаживающим счетом, чтобы избежать вычисления логарифма 0. 

Во внешней метрике каждое слово попарно сравнивается с каждым другим словом. Во внешней метрике используется PMI (Pointwise Mutual Information). Он рассчитвается как логорифм от деления числа документов, которые содержат оба слова, на произведения количества документов, содержащих слово 1, и количества документов, содержащих слово 2. 

Как внутренняя, так и внешняя метрика вычисляют показатель согласованности c (сумма попарных оценок слов w1,…, wn, используемых для описания темы).
